In [153]:
import numpy as np
from itertools import product
import importlib
import short_path_finding
import random
importlib.reload(short_path_finding)

# Import functions from the py file
from short_path_finding import (
    build_graph_from_matrix,
    euclidean_heuristic_factory,
    dijkstra_with_frontier,
    bellman_ford_with_frontier,
    astar_with_frontier,
)

def create_problem(
    size: int,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
):
    """
    Returns:
        cost_matrix : (size, size) np.ndarray with np.inf for 'no edge'
        coords      : (size, 2)  np.ndarray with city positions
    """
    rng = np.random.default_rng()

    # City coordinates
    coords = rng.random((size, 2))  # in [0,1]x[0,1]

    # Start with infinite costs (no edges)
    cost_matrix = np.full((size, size), np.inf, dtype=float)

    for i, j in product(range(size), repeat=2):
        if i == j:
            cost_matrix[i, j] = 0.0
            continue

        if rng.random() < density:
            dx = coords[i, 0] - coords[j, 0]
            dy = coords[i, 1] - coords[j, 1]
            base_dist = np.hypot(dx, dy)

            # noise
            if negative_values:
                noise = (rng.random() * 2 - 1) * noise_level  # [-noise, +noise]
            else:
                noise = rng.random() * noise_level           # [0, noise]

            cost_matrix[i, j] = base_dist + noise

    return cost_matrix, coords

In [154]:
# Problem parameters can be customize
size = 200
density = 1.0
noise_level = 0.5
negative_values = False  # True will activate Bellman-Ford only

cost_matrix, coords = create_problem(
    size=size,
    density=density,
    negative_values=negative_values,
    noise_level=noise_level,
    seed=42,
)

print("Cost matrix shape:", cost_matrix.shape)

Cost matrix shape: (200, 200)


In [155]:
G = build_graph_from_matrix(cost_matrix)
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

Number of nodes: 200
Number of edges: 39800


In [156]:
# Randomly select two cities (source and target)
# Use current time as seed to get different results each run
import time
rng = np.random.default_rng(int(time.time()) % 2**32)

nodes = list(G.nodes)
# Shuffle the nodes list in-place for display (shows random order)
rng.shuffle(nodes)

print(f"Full list of all nodes in the graph (shuffled):")
print(f"  {nodes}")
print(f"\nTotal number of nodes: {len(nodes)}")

# Randomly select two different nodes
A, B = rng.choice(nodes, size=2, replace=False)
A, B = int(A), int(B)  # Ensure they're integers

print(f"\nRandomly selected cities:")
print(f"  Source (A): {A} {'✓' if A in nodes else '✗'}")
print(f"  Target (B): {B} {'✓' if B in nodes else '✗'}")
print(f"\nNow computing shortest path using all 3 algorithms...")

Full list of all nodes in the graph (shuffled):
  [166, 81, 18, 51, 179, 134, 114, 80, 113, 38, 194, 133, 95, 64, 54, 53, 23, 184, 107, 172, 29, 199, 25, 183, 33, 16, 59, 161, 71, 122, 27, 26, 67, 101, 65, 96, 153, 135, 9, 34, 115, 3, 195, 180, 167, 190, 15, 97, 69, 140, 120, 144, 5, 20, 79, 77, 186, 93, 76, 94, 123, 160, 111, 105, 178, 175, 0, 2, 89, 139, 62, 87, 173, 57, 66, 128, 156, 78, 21, 126, 116, 162, 151, 147, 185, 181, 48, 182, 13, 127, 40, 91, 49, 106, 192, 19, 14, 100, 163, 174, 121, 154, 41, 84, 142, 30, 31, 149, 63, 22, 138, 112, 74, 10, 85, 58, 47, 170, 197, 70, 98, 43, 187, 6, 131, 118, 60, 110, 137, 136, 193, 165, 68, 82, 73, 61, 75, 88, 52, 7, 99, 189, 103, 150, 152, 12, 117, 36, 4, 130, 92, 86, 198, 177, 164, 42, 32, 50, 168, 176, 155, 46, 35, 39, 124, 145, 196, 146, 129, 11, 1, 132, 141, 45, 143, 188, 171, 169, 55, 56, 44, 28, 72, 191, 104, 90, 109, 157, 119, 108, 37, 158, 125, 8, 17, 102, 148, 159, 83, 24]

Total number of nodes: 200

Randomly selected cities:
  So

In [157]:
# Run Dijkstra algorithm
print("="*60)
print("1. DIJKSTRA ALGORITHM")
print("="*60)
dij_result = dijkstra_with_frontier(G, A, B)

dij_path = dij_result['path']
dij_cost = dij_result['cost']
dij_frontier = dij_result['frontier']
dij_nodes_explored = dij_result.get('nodes_explored', len(dij_frontier))
dij_status = dij_result['status']

print(f"Status: {dij_status}")
print(f"Cost: {dij_cost:.6f}")
print(f"Path: {dij_path}")
print(f"Path length: {len(dij_path)} nodes")
print(f"Frontier size: {len(dij_frontier)}")
print(f"Nodes explored: {dij_nodes_explored}")
print(f"Frontier: {dij_frontier[:15]}{'...' if len(dij_frontier) > 15 else ''}")

1. DIJKSTRA ALGORITHM
Status: ok
Cost: 0.688419
Path: [109, 195, 49, 131]
Path length: 4 nodes
Frontier size: 125
Nodes explored: 125
Frontier: [109, 195, 168, 128, 84, 42, 47, 89, 123, 76, 44, 21, 165, 178, 162]...


In [158]:
# Run Bellman-Ford algorithm
print("\n" + "="*60)
print("2. BELLMAN-FORD ALGORITHM")
print("="*60)
bf_result = bellman_ford_with_frontier(G, A, B)

bf_path = bf_result['path']
bf_cost = bf_result['cost']
bf_frontier = bf_result['frontier']
bf_nodes_explored = bf_result.get('nodes_explored', len(bf_frontier))
bf_status = bf_result['status']

print(f"Status: {bf_status}")
print(f"Cost: {bf_cost:.6f}")
print(f"Path: {bf_path}")
print(f"Path length: {len(bf_path)} nodes")
print(f"Frontier size: {len(bf_frontier)}")
print(f"Nodes explored: {bf_nodes_explored}")
print(f"Frontier: {bf_frontier[:15]}{'...' if len(bf_frontier) > 15 else ''}")


2. BELLMAN-FORD ALGORITHM
Status: ok
Cost: 0.688419
Path: [109, 195, 49, 131]
Path length: 4 nodes
Frontier size: 636
Nodes explored: 200
Frontier: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]...


In [159]:
# Run A* algorithm (only if no negative values)
if not negative_values:
    print("\n" + "="*60)
    print("3. A* ALGORITHM")
    print("="*60)
    heuristic = euclidean_heuristic_factory(coords, scale=1.0)
    astar_result = astar_with_frontier(G, A, B, heuristic)
    
    astar_path = astar_result['path']
    astar_cost = astar_result['cost']
    astar_frontier = astar_result['frontier']
    astar_nodes_explored = astar_result.get('nodes_explored', len(astar_frontier))
    astar_status = astar_result['status']
    
    print(f"Status: {astar_status}")
    print(f"Cost: {astar_cost:.6f}")
    print(f"Path: {astar_path}")
    print(f"Path length: {len(astar_path)} nodes")
    print(f"Frontier size: {len(astar_frontier)}")
    print(f"Nodes explored: {astar_nodes_explored}")
    print(f"Frontier: {astar_frontier[:15]}{'...' if len(astar_frontier) > 15 else ''}")
else:
    print("\n" + "="*60)
    print("3. A* ALGORITHM")
    print("="*60)
    print("Skipping A* because negative values are present")
    astar_result = None
    astar_path = []
    astar_cost = np.inf
    astar_frontier = []
    astar_nodes_explored = 0
    astar_status = "skipped"


3. A* ALGORITHM
Status: ok
Cost: 0.688419
Path: [109, 195, 49, 131]
Path length: 4 nodes
Frontier size: 18
Nodes explored: 18
Frontier: [109, 128, 182, 104, 195, 174, 177, 44, 157, 138, 178, 26, 17, 49, 156]...


In [160]:
# Comparison Summary
print("\n" + "="*60)
print("COMPARISON SUMMARY")
print("="*60)

print(f"\nSource: {A}, Target: {B}\n")

# Create comparison table
print(f"{'Algorithm':<15} {'Status':<15} {'Cost':<12} {'Path Length':<12} {'Nodes Explored':<15}")
print("-" * 70)

# Dijkstra
print(f"{'Dijkstra':<15} {dij_status:<15} {dij_cost:<12.6f} {len(dij_path):<12} {dij_nodes_explored:<15}")

# Bellman-Ford
bf_cost_str = f"{bf_cost:.6f}" if bf_status != 'negative_cycle' else "N/A (cycle)"
print(f"{'Bellman-Ford':<15} {bf_status:<15} {bf_cost_str:<12} {len(bf_path):<12} {bf_nodes_explored:<15}")

# A*
if not negative_values:
    print(f"{'A*':<15} {astar_status:<15} {astar_cost:<12.6f} {len(astar_path):<12} {astar_nodes_explored:<15}")
else:
    print(f"{'A*':<15} {'skipped':<15} {'N/A':<12} {'N/A':<12} {'N/A':<15}")

# Verify all algorithms found the same cost (if applicable)
print("\n" + "-" * 70)
if not negative_values and dij_status == 'ok' and bf_status == 'ok' and astar_status == 'ok':
    costs_match = (abs(dij_cost - bf_cost) < 1e-6 and 
                   abs(dij_cost - astar_cost) < 1e-6 and
                   abs(bf_cost - astar_cost) < 1e-6)
    paths_match = (dij_path == bf_path == astar_path)
    
    print(f"✓ All algorithms agree on cost: {costs_match}")
    print(f"✓ All algorithms found the same path: {paths_match}")
    if costs_match:
        print(f"  Best cost: {dij_cost:.6f}")
    if paths_match:
        print(f"  Best path: {dij_path}")
elif dij_status == 'ok' and bf_status == 'ok':
    costs_match = abs(dij_cost - bf_cost) < 1e-6
    paths_match = (dij_path == bf_path)
    print(f"✓ Dijkstra and Bellman-Ford agree on cost: {costs_match}")
    print(f"✓ Dijkstra and Bellman-Ford found the same path: {paths_match}")
    if costs_match:
        print(f"  Best cost: {dij_cost:.6f}")
    if paths_match:
        print(f"  Best path: {dij_path}")


COMPARISON SUMMARY

Source: 109, Target: 131

Algorithm       Status          Cost         Path Length  Nodes Explored 
----------------------------------------------------------------------
Dijkstra        ok              0.688419     4            125            
Bellman-Ford    ok              0.688419     4            200            
A*              ok              0.688419     4            18             

----------------------------------------------------------------------
✓ All algorithms agree on cost: True
✓ All algorithms found the same path: True
  Best cost: 0.688419
  Best path: [109, 195, 49, 131]
